In [0]:
%python
dbutils.widgets.text("catalog", defaultValue="production")
dbutils.widgets.text("schema", defaultValue="finance")
dbutils.widgets.text("uc_service_credential", defaultValue="production-akv")

In [0]:
SELECT session_user() AS current_user

### Step 1
* Check whether the user is member of the relevant account level group

In [0]:
SELECT is_account_group_member(concat(:catalog, '.', :schema, '.crypto.user')) AS is_allowed_to_decrypt

### Step 2
* Check that the user cannot access the raw data

In [0]:
SELECT * 
FROM read_files(
  concat('/Volumes/', :catalog, '/', :schema, '/raw_files/titanic.csv'),
  format => 'csv',
  header => true,
  mode => 'FAILFAST');

### Step 3
* Check that the user cannot view or access the `crypto` schema

In [0]:
USE CATALOG IDENTIFIER(:catalog);

In [0]:
SHOW SCHEMAS;

In [0]:
SHOW FUNCTIONS IN IDENTIFIER(concat(:catalog, '.crypto'))

### Step 4
* Check that the user cannot access the UC service credential

In [0]:
--Todo!

### Step 5
* Check that the user cannot access any of the crypto functions

In [0]:
DESCRIBE FUNCTION EXTENDED IDENTIFIER(concat(:catalog, '.crypto.unwrap_akv_key'));

In [0]:
DESCRIBE FUNCTION EXTENDED IDENTIFIER(concat(:catalog, '.crypto.get_encrypted_dek'));

In [0]:
DESCRIBE FUNCTION EXTENDED IDENTIFIER(concat(:catalog, '.crypto.encrypt_akv'));

In [0]:
DESCRIBE FUNCTION EXTENDED IDENTIFIER(concat(:catalog, '.crypto.decrypt_akv'));

In [0]:
DESCRIBE FUNCTION EXTENDED IDENTIFIER(concat(:catalog, '.crypto.encrypt'));

In [0]:
DESCRIBE FUNCTION EXTENDED IDENTIFIER(concat(:catalog, '.crypto.decrypt'));

In [0]:
DESCRIBE FUNCTION EXTENDED IDENTIFIER(concat(:catalog, '.crypto.get_encrypted_dek'));

In [0]:
SELECT crypto.get_encrypted_dek(:catalog, :schema)

In [0]:
SELECT crypto.encrypt("This should fail", :catalog, :schema) AS this_should_fail

### Step 6
* Check whether the user can access the data

In [0]:
SELECT *
FROM IDENTIFIER(:catalog || '.' || :schema || '.titanic');

In [0]:
DESCRIBE TABLE EXTENDED IDENTIFIER(:catalog || '.' || :schema || '.titanic');

### Step 7
* Now add the user to the the decrypt group shown below, wait ~5 minutes for the groups to sync fully and then run the following command to check whether the user can decrypt the data. 
* Optionally repeat the steps above to ensure that they still cannot access the crypto schema or functions 

In [0]:
SELECT concat(:catalog, '.', :schema, '.crypto.user') AS group_to_add_user_to

In [0]:
SELECT is_account_group_member(concat(:catalog, '.', :schema, '.crypto.user')) AS is_allowed_to_decrypt;

In [0]:
SELECT *
FROM IDENTIFIER(:catalog || '.' || :schema || '.titanic');